In [ ]:
_articles = "../../../Data/sumdata/train/train.article.txt"
_titles = "../../../Data/sumdata/train/train.title.txt"

In [ ]:
articles = []
titles = []
with open(_articles, "r") as f:
    articles = f.read().split("\n")
with open(_titles, "r") as f:
    titles = f.read().split("\n")
print(len(articles), len(titles))

In [ ]:
from pytorch_pretrained_bert import BertTokenizer
import numpy as np
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
cls_token = tokenizer.convert_tokens_to_ids(["[CLS]"])[0]
print(cls_token)

In [ ]:
max_doc_len = 100
max_sum_len = 20

In [ ]:
dps = []

In [ ]:
for i in range(len(articles[:1000000])):
    _article = None
    _summary = None
    _article_ids = None
    _summary_ids = None
    _article = tokenizer.tokenize(articles[i])
    _summary = tokenizer.tokenize(titles[i])
    _article_ids = tokenizer.convert_tokens_to_ids(_article)
    _summary_ids = tokenizer.convert_tokens_to_ids(_summary)
    _article_ids.insert(0, cls_token)
    while (len(_article_ids) < max_doc_len):
        _article_ids.append(0)
        
    _article_ids = _article_ids[:max_doc_len]
    while (len(_summary_ids) < max_sum_len):
        _summary_ids.append(0)
    _summary_ids = _summary_ids[:max_sum_len]
    pointers = []
    for k in range(len(_summary)):
        token = _summary[k]
        if (any(char.isdigit() for char in token)):
            pointers.append(k)
    dp = {
        "story":_article,
        "summary":_summary,
        "story_tokens":_article_ids,
        "summary_tokens":_summary_ids,
        "pointers":pointers
    }
    dps.append(dp)
    _str = str(i+1) + "/" + str(len(articles))
    print(_str, end="\r")

In [ ]:
import pickle
import pprint
step = 10

f_name = "../../../Data/sumdata/training_0.pickle"
pickle.dump(dps, open(f_name, "wb"))

In [ ]:
import pprint
pprint.pprint(dps)

In [ ]:
def genBatch(bs = 64, max_article_len = 100, max_summary_len = 15):
    indices = np.random.randint(0, len(articles), (bs,))
    _docs = [tokenizer.tokenize(articles[index]) for index in indices]
    _sums = [tokenizer.tokenize(titles[index]) for index in indices]
    for doc in _docs:
        doc.insert(0, "[CLS]")
        doc.append("[SEP]")
        while (len(doc) < max_doc_len):
            doc.append("[PAD]")
        doc = doc[:max_doc_len]
    
    copy = []
    for summary in _sums:
        doc = summary
        while (len(doc) < max_sum_len):
            doc.append("[PAD]")
        doc = doc[:max_sum_len]
    
    _docs = [tokenizer.convert_tokens_to_ids(doc) for doc in _docs]
    _sums = [tokenizer.convert_tokens_to_ids(summary) for summary in _sums]
        
genBatch(bs=5)